In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import json
from PIL import Image
from collections import Counter
import pickle

In [2]:
base_path = '../'
tile_categories = ['blood_vessel', 'glomerulus', 'unsure']
input_imgs_dir = f'{base_path}/dataset1_files/all_dataset1_imgs'
output_imgs_dir = f'{base_path}/dataset1_files/all_dataset1_imgs_merged'
vis_output_imgs_dir = f'{base_path}/dataset1_files/all_dataset1_vis'
output_annots_dir = f'{base_path}/dataset1_files/all_dataset1_annotations_merged'

if not os.path.exists(output_imgs_dir):
  os.mkdir(output_imgs_dir)
if not os.path.exists(vis_output_imgs_dir):
  os.mkdir(vis_output_imgs_dir)
if not os.path.exists(output_annots_dir):
  os.mkdir(output_annots_dir)

In [3]:
def get_count_type(tiles_dict):
  ids_with_info = []
  for tile in tiles_dict:
    cur_dict = {'id': tile['id'], 'blood_vessel': 0, 'glomerulus': 0, 'unsure': 0}
    for annot in tile['annotations']:
      cur_dict[annot['type']] += 1
    ids_with_info.append(cur_dict)
  return ids_with_info
def calculate_area(coordinates):
    num_points = len(coordinates)
    if num_points < 3:
        return 0

    area = 0
    for i in range(num_points - 1):
        x_i, y_i = coordinates[i]
        x_iplus1, y_iplus1 = coordinates[i + 1]
        area += (x_i * y_iplus1) - (x_iplus1 * y_i)

    x_n, y_n = coordinates[-1]
    x_0, y_0 = coordinates[0]
    area += (x_n * y_0) - (x_0 * y_n)

    area = abs(area / 2)
    return area
# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(30, 30))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()

In [4]:
with open(f'{base_path}/polygons.jsonl', 'r') as json_file:
    json_list = list(json_file)
    
tiles_dicts = []
for json_str in json_list:
    tiles_dicts.append(json.loads(json_str))

In [5]:
tile_df = pd.read_csv(f'{base_path}/tile_meta.csv')
# Find the same stats as the above cell for annotated images
annotated_ids_with_info = get_count_type(tiles_dicts)
annotated_ids_with_info_df = pd.DataFrame.from_dict(annotated_ids_with_info)
tile_df_annotated = pd.merge(tile_df, annotated_ids_with_info_df, on='id', how='inner')
tile_df_annotated.head()

,id,source_wsi,dataset,i,j,blood_vessel,glomerulus,unsure
0,0006ff2aa7cd,2,2,16896,16420,8,1,0
1,00168d1b7522,2,2,14848,14884,1,1,0
2,0033bbc76b6b,1,1,10240,43008,3,0,1
3,003504460b3a,3,2,8192,11776,7,0,0
4,004daf1cbe75,3,2,6144,11264,10,1,0


In [6]:
dataset_1 = tile_df_annotated.loc[tile_df_annotated['dataset']==1]
dataset_1_tile_ids = sorted(dataset_1.loc[:, 'id'].values)
dataset_1_wsi_1_pos_dict = {(x[0], x[1]): x[-1] for x in dataset_1.loc[dataset_1['source_wsi']==1, ['i', 'j', 'id']].values}
dataset_1_wsi_2_pos_dict = {(x[0], x[1]): x[-1] for x in dataset_1.loc[dataset_1['source_wsi']==2, ['i', 'j', 'id']].values}
reverse_dataset_1_wsi_1_pos_dict = {v:k for k,v in dataset_1_wsi_1_pos_dict.items()}
reverse_dataset_1_wsi_2_pos_dict = {v:k for k,v in dataset_1_wsi_2_pos_dict.items()}

In [7]:
import math
## Below is the code to create variability by merging adjacent tiles
def get_relevant_annots(tgt_tile_dicts):
  final_tile_dicts = dict()
  for tile_dict in tgt_tile_dicts:
    img_id = tile_dict['id']
    final_annots = []
    for annot in tile_dict['annotations']:
      if annot['type'] in tile_categories:
        final_annots.append((annot['coordinates'][0], tile_categories.index(annot['type'])))
    final_tile_dicts[img_id] = final_annots
  return final_tile_dicts
def get_adjacent_tiles(i, j, img_id, adjacent_tile_dict, dataset_1_pos_dict):
  adjacent_tile_dict[img_id] = dict()
  keys = ['B', 'L', 'R', 'BL', 'BR']
  for k in keys:
    tgt_img_id = None
    if k == 'B':
      pos_to_check = (i, j+512)
    elif k == 'L':
      pos_to_check = (i-512, j)
    elif k == 'R':
      pos_to_check = (i+512, j)
    elif k == 'BL':
      pos_to_check = (i-512, j+512)
    elif k == 'BR':
      pos_to_check = (i+512, j+512)
    if pos_to_check in dataset_1_pos_dict:
      tgt_img_id = dataset_1_pos_dict[pos_to_check]
    adjacent_tile_dict[img_id][k] = tgt_img_id
  return adjacent_tile_dict
def single_shift_polygon(seg_array_1, seg_array_2, shifting_threshold, shift_type, border_eps=1):
    polygon_regions = []
    num_polygons_1 = len(seg_array_1)
    num_polygons_2 = len(seg_array_2)
    
    combined_array_1 = []
    combined_array_2 = []
    for i in range(num_polygons_1):
        polygon_1,category_id = seg_array_1[i]
        new_polygon = []
        for point in polygon_1:
            x, y = point
            if shift_type == 'R':
              if x >= math.ceil(shifting_threshold*512):
                  new_x = x - int(math.ceil(shifting_threshold*512))
                  new_y = y
                  new_polygon.append([new_x, new_y])
            elif shift_type == 'B':
              if y >= math.ceil(shifting_threshold*512):
                new_x = x
                new_y = y - int(math.ceil(shifting_threshold*512))
                new_polygon.append([new_x, new_y])
            elif shift_type == 'L':
              if x <= math.floor((1-shifting_threshold)*512):
                new_x = x + int(math.ceil(shifting_threshold*512))
                new_y = y
                new_polygon.append([new_x, new_y])
        if len(new_polygon) > 0:
          combined_array_1.append((new_polygon, category_id))
    
    polygon_regions.append(combined_array_1)
    
    for i in range(num_polygons_2):
        polygon_2, category_id = seg_array_2[i]
        new_polygon = []
        for point in polygon_2:
            x, y = point
            if shift_type == 'R':
              if x <= math.ceil(shifting_threshold*512):
                new_x = x + int(math.floor((1-shifting_threshold)*512))
                new_y = y
                new_polygon.append([new_x, new_y])
            elif shift_type == 'B':
              if y <= math.ceil(shifting_threshold*512):
                new_x = x
                new_y = y + int(math.floor((1-shifting_threshold)*512))
                new_polygon.append([new_x, new_y])
            elif shift_type == 'L':
              if x >= math.floor((1-shifting_threshold)*512):
                  new_x = x - int(math.floor((1-shifting_threshold)*512))
                  new_y = y
                  new_polygon.append([new_x, new_y])
        if len(new_polygon) > 0:
          combined_array_2.append((new_polygon, category_id))
    
    polygon_regions.append(combined_array_2)
    
    return polygon_regions

def double_shift_polygon(seg_array_1, seg_array_2, seg_array_3, seg_array_4, shifting_threshold, shift_type, border_eps=1):
  polygon_regions = []
  num_polygons_1 = len(seg_array_1)
  num_polygons_2 = len(seg_array_2)
  num_polygons_3 = len(seg_array_3)
  num_polygons_4 = len(seg_array_4)
  
  combined_array = []
  for i in range(num_polygons_1):
    polygon_1, category_id = seg_array_1[i]
    new_polygon = []
    for point in polygon_1:
        x, y = point
        if shift_type == 'BR':
          if x >= math.ceil(shifting_threshold*512) and y >= math.ceil(shifting_threshold*512):
              new_x = x - int(math.ceil(shifting_threshold*512))
              new_y = y - int(math.ceil(shifting_threshold*512))
              new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x <= math.floor((1-shifting_threshold)*512) and y >= math.ceil(shifting_threshold*512):
            new_x = x + int(math.ceil(shifting_threshold*512))
            new_y = y - int(math.ceil(shifting_threshold*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append((new_polygon, category_id))
  
  polygon_regions.append(combined_array)
  
  combined_array = []
  for i in range(num_polygons_2):
    polygon_2, category_id = seg_array_2[i]
    new_polygon = []
    for point in polygon_2:
        x, y = point
        if shift_type == 'BR':
          if x <= math.ceil(shifting_threshold*512) and y >= math.ceil(shifting_threshold*512):
              new_x = x + int(math.floor((1-shifting_threshold)*512))
              new_y = y - int(math.ceil(shifting_threshold*512))
              new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x >= math.floor((1-shifting_threshold)*512) and y >= math.ceil(shifting_threshold*512):
            new_x = x - int(math.floor((1-shifting_threshold)*512))
            new_y = y - int(math.ceil(shifting_threshold*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append((new_polygon, category_id))
    
  polygon_regions.append(combined_array)
  
  combined_array = []
  for i in range(num_polygons_3):
    polygon_3, category_id = seg_array_3[i]
    new_polygon = []
    for point in polygon_3:
        x, y = point
        if shift_type == 'BR':
          if x >= math.ceil(shifting_threshold*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x - int(math.ceil(shifting_threshold*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x <= math.floor((1-shifting_threshold)*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x + int(math.ceil(shifting_threshold*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append((new_polygon, category_id))
  
  polygon_regions.append(combined_array)
  
  combined_array = []
  for i in range(num_polygons_4):
    polygon_4, category_id = seg_array_4[i]
    new_polygon = []
    for point in polygon_4:
        x, y = point
        if shift_type == 'BR':
          if x <= math.ceil(shifting_threshold*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x + int(math.floor((1-shifting_threshold)*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
        elif shift_type == 'BL':
          if x >= math.floor((1-shifting_threshold)*512) and y <= math.ceil(shifting_threshold*512):
            new_x = x - int(math.floor((1-shifting_threshold)*512))
            new_y = y + int(math.floor((1-shifting_threshold)*512))
            new_polygon.append([new_x, new_y])
    if len(new_polygon) > 0:
      combined_array.append((new_polygon, category_id))
  
  polygon_regions.append(combined_array)
  
  return polygon_regions

def check_polygon_border(pt, border_eps=1):
  return pt[0] <= border_eps or pt[0] >= 512-border_eps or pt[1] <= border_eps or pt[1] >= 512-border_eps

def single_shift_img(img_1, img_2, shifting_threshold, shift_type):
    height, width, _ = img_1.shape
    
    if shift_type == 'R':
      img_1_part = img_1[:, int(math.ceil(shifting_threshold*width)):, :]
      img_2_part = img_2[:, :int(math.ceil(shifting_threshold*width)), :]
      combined_image = cv2.hconcat([img_1_part, img_2_part])
    elif shift_type == 'B':
      img_1_part = img_1[int(math.ceil(shifting_threshold*height)):, :, :]
      img_2_part = img_2[:int(math.ceil(shifting_threshold*height)), :, :]
      combined_image = cv2.vconcat([img_1_part, img_2_part])
    else:
      img_1_part = img_1[:, :int(math.floor((1-shifting_threshold)*width)), :]
      img_2_part = img_2[:, int(math.floor((1-shifting_threshold)*width)):, :]
      combined_image = cv2.hconcat([img_2_part, img_1_part])
    return combined_image

def double_shift_img(img_1, img_2, img_3, img_4, shifting_threshold, shift_type):
  height, width, _ = img_1.shape
  if shift_type == 'BR':
    img_1_part = img_1[int(math.ceil(shifting_threshold*height)):, int(math.ceil(shifting_threshold*width)):, :]
    img_2_part = img_2[int(math.ceil(shifting_threshold*height)):, :int(math.ceil(shifting_threshold*width)), :]
    img_3_part = img_3[:int(math.ceil(shifting_threshold*height)), int(math.ceil(shifting_threshold*width)):, :]
    img_4_part = img_4[:int(math.ceil(shifting_threshold*height)), :int(math.ceil(shifting_threshold*width)), :]
    image_top = cv2.hconcat([img_1_part, img_2_part])
    image_bottom = cv2.hconcat([img_3_part, img_4_part])
    combined_image = cv2.vconcat([image_top, image_bottom])
  else:
    img_1_part = img_1[int(math.ceil(shifting_threshold*height)):, :int(math.floor((1-shifting_threshold)*width)), :]
    img_2_part = img_2[int(math.ceil(shifting_threshold*height)):, int(math.floor((1-shifting_threshold)*width)):, :]
    img_3_part = img_3[:int(math.ceil(shifting_threshold*height)), :int(math.floor((1-shifting_threshold)*width)), :]
    img_4_part = img_4[:int(math.ceil(shifting_threshold*height)), int(math.floor((1-shifting_threshold)*width)):, :]
    image_top = cv2.hconcat([img_2_part, img_1_part])
    image_bottom = cv2.hconcat([img_4_part, img_3_part])
    combined_image = cv2.vconcat([image_top, image_bottom])
  return combined_image

def load_img(img_dir, img_id):
  img = cv2.imread(f"{img_dir}/{img_id}.png")
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

def add_border(image, border_size, border_color):
    height, width = image.shape[:2]
    new_height = height + 2 * border_size
    new_width = width + 2 * border_size
    bordered_image = np.zeros((new_height, new_width, 3), dtype=np.uint8)
    bordered_image[border_size:height + border_size, border_size:width + border_size] = image
    cv2.rectangle(bordered_image, (0, 0), (new_width - 1, new_height - 1), border_color, border_size)
    return bordered_image

def create_coco_annots(all_coords, pair_format=True):
  cur_tile_coco_annots = []
  for coords,category_id in all_coords:
#     print('======================')
#     print(coords)
#     print('======================')
    if not pair_format:
      coords = [[pt[0], pt[1]] for pt in zip(coords[::2], coords[1::2])]
    segmentations = [[pt for pair in coords for pt in pair]]
    segmentation_area = calculate_area(coords)
    min_x = min(coords, key=lambda x: x[0])[0]
    max_x = max(coords, key=lambda x: x[0])[0]
    min_y = min(coords, key=lambda x: x[1])[1]
    max_y = max(coords, key=lambda x: x[1])[1]
    segmentation_bbox = [min_x, min_y, max_x-min_x, max_y-min_y]
    cur_tile_coco_annots.append({
      'segmentation': segmentations,
      'area': segmentation_area,
      'bbox': segmentation_bbox,
      'category_id': category_id
    })
  return cur_tile_coco_annots

def save_merged_imgs(input_imgs_dir, output_imgs_dir, base_img_id, base_img_dict, shifting_thresholds):
  img_arr_dict = {'base_img': load_img(input_imgs_dir, base_img_id)}
  cv2.imwrite(f'{output_imgs_dir}/{base_img_id}.png', img_arr_dict['base_img'])
  for shift_type in base_img_dict.keys():
    if base_img_dict[shift_type] is not None:
      img_arr_dict[shift_type] = load_img(input_imgs_dir, base_img_dict[shift_type])
    else:
      img_arr_dict[shift_type] = None
  for shifting_threshold in shifting_thresholds:
    for shift_type in img_arr_dict:
      if img_arr_dict[shift_type] is not None:
        if shift_type in ['R', 'B']:
          shifted_img = single_shift_img(img_arr_dict['base_img'], img_arr_dict[shift_type], shifting_threshold, shift_type)
          if shifted_img.shape[0] != 512 or shifted_img.shape[1] != 512 or shifted_img.shape[2] != 3:
            print(f'Error!!! Got an odd combined shape: {shifted_img.shape} for image id: {base_img_id} and shift type: {shift_type} and shifting threshold: {shifting_threshold}')
          cv2.imwrite(f'{output_imgs_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.png', shifted_img)
        elif shift_type == 'BR' and img_arr_dict['B'] is not None and img_arr_dict['R'] is not None:
          shifted_img = double_shift_img(img_arr_dict['base_img'], img_arr_dict['R'], img_arr_dict['B'], img_arr_dict[shift_type], shifting_threshold, shift_type)
          if shifted_img.shape[0] != 512 or shifted_img.shape[1] != 512 or shifted_img.shape[2] != 3:
            print(f'Error!!! Got an odd combined shape: {shifted_img.shape} for image id: {base_img_id} and shift type: {shift_type}')
          cv2.imwrite(f'{output_imgs_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.png', shifted_img)
        elif shift_type == 'BL' and img_arr_dict['B'] is not None and img_arr_dict['L'] is not None:
          shifted_img = double_shift_img(img_arr_dict['base_img'], img_arr_dict['L'], img_arr_dict['B'], img_arr_dict[shift_type], shifting_threshold, shift_type)
          if shifted_img.shape[0] != 512 or shifted_img.shape[1] != 512 or shifted_img.shape[2] != 3:
            print(f'Error!!! Got an odd combined shape: {shifted_img.shape} for image id: {base_img_id} and shift type: {shift_type}')
          cv2.imwrite(f'{output_imgs_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.png', shifted_img)

def save_merged_annotations(output_annots_dir, base_img_id, base_img_dict, shifting_thresholds, tgt_tile_dicts):
  print(f'base image id: {base_img_id}')
  cur_tile_annots = create_coco_annots(tgt_tile_dicts[base_img_id])
  with open(f'{output_annots_dir}/{base_img_id}.pkl', 'wb') as f:
    pickle.dump(cur_tile_annots, f, protocol=pickle.HIGHEST_PROTOCOL)
  for shifting_threshold in shifting_thresholds:
    for shift_type in base_img_dict:
      if base_img_dict[shift_type] is not None:
        if shift_type in ['R', 'B']:
          polygon_regions = single_shift_polygon(tgt_tile_dicts[base_img_id], tgt_tile_dicts[base_img_dict[shift_type]], shifting_threshold, shift_type)
#           print(f'===========after single shift - {len(polygon_regions)}============')
#           print(polygon_regions)
#           print('===========after single shift============')
          shifted_coords = get_polygon_merged_coords(polygon_regions)
#           print(f'===========after polyon merge - {len(shifted_coords)}============')
#           print(shifted_coords)
#           print('===========after polyon merge============')
#           print(f'===========merged coords shift type R B===========')
#           print(shifted_coords)
#           print(f'===========merged coords shift type R B===========')
          shifted_annots = create_coco_annots(shifted_coords, pair_format=False)
#           print(f'===========after coco - {len(shifted_coords)}============')
#           print(shifted_annots)
#           print('===========after coco============')
          with open(f'{output_annots_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.pkl', 'wb') as f:
            pickle.dump(shifted_annots, f, protocol=pickle.HIGHEST_PROTOCOL)
        elif shift_type == 'BR' and base_img_dict['B'] is not None and base_img_dict['R'] is not None:
          polygon_regions = double_shift_polygon(tgt_tile_dicts[base_img_id], tgt_tile_dicts[base_img_dict['R']], tgt_tile_dicts[base_img_dict['B']], tgt_tile_dicts[base_img_dict[shift_type]], shifting_threshold, shift_type)
          shifted_coords = get_polygon_merged_coords(polygon_regions)
#           print(f'===========merged coords shift type BR===========')
#           print(shifted_coords)
#           print(f'===========merged coords shift type BR===========')
          shifted_annots = create_coco_annots(shifted_coords, pair_format=False)
          with open(f'{output_annots_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.pkl', 'wb') as f:
            pickle.dump(shifted_annots, f, protocol=pickle.HIGHEST_PROTOCOL)
        elif shift_type == 'BL' and base_img_dict['B'] is not None and base_img_dict['L'] is not None:
          polygon_regions = double_shift_polygon(tgt_tile_dicts[base_img_id], tgt_tile_dicts[base_img_dict['L']], tgt_tile_dicts[base_img_dict['B']], tgt_tile_dicts[base_img_dict[shift_type]], shifting_threshold, shift_type)
          shifted_coords = get_polygon_merged_coords(polygon_regions)
#           print(f'===========merged coords shift type BL===========')
#           print(shifted_coords)
#           print(f'===========merged coords shift type BL===========')
          shifted_annots = create_coco_annots(shifted_coords, pair_format=False)
          with open(f'{output_annots_dir}/{base_img_id}_{shift_type}_{shifting_threshold}.pkl', 'wb') as f:
            pickle.dump(shifted_annots, f, protocol=pickle.HIGHEST_PROTOCOL)

def get_annotated_img(imgs_dir, annots_dir, base_img_id):
  eps = 1
  annotated_image_coco = load_img(imgs_dir, base_img_id)
  img_height, img_width, _ = annotated_image_coco.shape
  with open(f'{annots_dir}/{base_img_id}.pkl', 'rb') as f:
    tgt_annots = pickle.load(f)
  for tgt_annot in tgt_annots:
    coords = [[x, y] for x, y in zip(tgt_annot['segmentation'][0][::2], tgt_annot['segmentation'][0][1::2])]
    min_x, min_y, width, height = tgt_annot['bbox']
    cv2.fillPoly(annotated_image_coco, pts=[np.array(coords)], color=(0,255,0))
    top_left = (int(tgt_annot['bbox'][0]), int(tgt_annot['bbox'][1]))
    min_x = min(coords, key=lambda x: x[0])[0]
    min_y = min(coords, key=lambda x: x[1])[1]
    max_x = max(coords, key=lambda x: x[0])[0]
    max_y = max(coords, key=lambda x: x[1])[1]
    bottom_right = (int(tgt_annot['bbox'][0]+tgt_annot['bbox'][2]), int(tgt_annot['bbox'][1]+tgt_annot['bbox'][3]))
#     if min_x <= eps or max_x >= img_width-eps or min_y <= eps or max_y >= img_height-eps:
#       annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
    annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
  annotated_image_coco = add_border(annotated_image_coco, 5, (0,0,255))
  return annotated_image_coco

def get_polygon_merged_annotated_img(imgs_dir, annots_dir, base_img_id):
  eps = 1
  annotated_image_coco = load_img(imgs_dir, base_img_id)
  img_height, img_width, _ = annotated_image_coco.shape
  annotated_image_mask = np.zeros((img_height,img_width))
  with open(f'{annots_dir}/{base_img_id}.pkl', 'rb') as f:
    tgt_annots = pickle.load(f)
  all_polygons = [x for tgt_annot in tgt_annots for x in tgt_annot['segmentation']]
#   all_polygons = merge_connected_polygons(all_polygons, 1)
  for polygon in all_polygons:
    coords = [[x, y] for x, y in zip(polygon[::2], polygon[1::2])]
    cv2.fillPoly(annotated_image_mask, pts=[np.array(coords)], color=255)
  merged_polygons = get_segmentation_coordinates(annotated_image_mask)
  for merged_polygon in merged_polygons:
    coords = [[x, y] for x, y in zip(merged_polygon[::2], merged_polygon[1::2])]
    cv2.fillPoly(annotated_image_coco, pts=[np.array(coords)], color=(0,255,0))
    min_x = min(coords, key=lambda x: x[0])[0]
    min_y = min(coords, key=lambda x: x[1])[1]
    max_x = max(coords, key=lambda x: x[0])[0]
    max_y = max(coords, key=lambda x: x[1])[1]
    top_left = (int(min_x), int(min_y))
    bottom_right = (int(max_x), int(max_y))
#     if min_x <= eps or max_x >= img_width-eps or min_y <= eps or max_y >= img_height-eps:
#       annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
    annotated_image_coco = cv2.rectangle(annotated_image_coco, top_left, bottom_right, (255,0,0), 2)
  annotated_image_coco = add_border(annotated_image_coco, 5, (0,0,255))
  return annotated_image_coco

def get_polygon_merged_coords(polygons_regions, epsilon=1):
  merged_polygons = group_merged_polygons(polygons_regions, epsilon=epsilon)
  final_segmentation_coords = []
  
  for merged_polygon, category_id in merged_polygons:
    annotated_image_mask = np.zeros((512,512))
    for coords in merged_polygon:
      cv2.fillPoly(annotated_image_mask, pts=[np.array(coords)], color=255)
#     plt.imshow(annotated_image_mask)
#     plt.show()
    final_segmentation_coords += get_segmentation_coordinates(annotated_image_mask, category_id)
  
  return final_segmentation_coords
  
import math

def group_merged_polygons(regions, epsilon):
    merged_groups = []
    
    def merge_polygons(poly1, poly2):
        return poly1 + poly2
    
    def distance(point1, point2):
        return math.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)
    
    def are_polygons_close(poly1, poly2):
        for p1 in poly1:
            for p2 in poly2:
                if distance(p1, p2) <= epsilon:
                    return True
        return False
    
    for region_index, region in enumerate(regions):
        for polygon, category_id in region:
            merged = False
            for group in merged_groups:
                if (not any(region_index == group_region_idx for group_region_idx, _ in group[0])) and any(are_polygons_close(polygon, poly) for _, poly in group[0]) and category_id==group[1]:
                    group[0].append([region_index, polygon])
                    merged = True
                    break
            if not merged:
                merged_groups.append(([[region_index, polygon]], category_id))
    
    merged_polygons = []
    for group, category_id in merged_groups:
        polygon_group = []
        for _, polygon in group:
          polygon_group.append(polygon)
        merged_polygons.append((polygon_group, category_id))
    return merged_polygons


def get_segmentation_coordinates(mask, category_id):
    mask = np.uint8(mask)
    
    # Find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Extract segmentation coordinates from the contours
    segmentation_coords = []
    for contour in contours:
        contour = np.squeeze(contour)
        coords = contour.flatten().tolist()
        segmentation_coords.append((coords, category_id))

    return segmentation_coords

def save_merged_img_id_with_annot(output_imgs_dir, output_annots_dir, vis_output_imgs_dir, base_img_id, base_img_dict, shifting_thresholds):
  img_arr_dict = {'base_image': get_annotated_img(output_imgs_dir, output_annots_dir, base_img_id)}
  for shift_type in base_img_dict.keys():
    if base_img_dict[shift_type] is not None:
      img_arr_dict[shift_type] = get_annotated_img(output_imgs_dir, output_annots_dir, base_img_dict[shift_type])
    else:
      img_arr_dict[shift_type] = add_border((np.zeros((512, 512, 3), dtype="uint8")), 5, (0,0,255))
  image_top = cv2.hconcat([img_arr_dict['L'], img_arr_dict['base_image'], img_arr_dict['R']])
  print(img_arr_dict['BL'].shape, img_arr_dict['B'].shape, img_arr_dict['BR'].shape)
  image_bottom = cv2.hconcat([img_arr_dict['BL'], img_arr_dict['B'], img_arr_dict['BR']])
  image_full = cv2.vconcat([image_top, image_bottom])
  cv2.imwrite(f'{vis_output_imgs_dir}/{base_img_id}_full_annotated.png', image_full)
  all_merged_img_ids = [x.split('.png')[0] for x in os.listdir(output_imgs_dir) if base_img_id in x and x != f'{base_img_id}.png']
  for merged_img_id in all_merged_img_ids:
    merged_annotated_img = get_annotated_img(output_imgs_dir, output_annots_dir, merged_img_id)
    cv2.imwrite(f'{vis_output_imgs_dir}/{merged_img_id}_full_annotated.png', merged_annotated_img)
#     polygon_merged_annotated_img = get_polygon_merged_annotated_img(output_imgs_dir, output_annots_dir, merged_img_id)
#     cv2.imwrite(f'{vis_output_imgs_dir}/{merged_img_id}_full_annotated_polygon_merged.png', polygon_merged_annotated_img)

In [8]:
tgt_tile_dicts = [x for x in tiles_dicts if x['id'] in dataset_1_tile_ids]
tgt_tile_dicts = get_relevant_annots(tgt_tile_dicts)
print(len(tgt_tile_dicts.keys()))
adjacent_tile_dict = dict()
for i,j in dataset_1_wsi_1_pos_dict.keys():
  adjacent_tile_dict = get_adjacent_tiles(i, j, dataset_1_wsi_1_pos_dict[(i,j)], adjacent_tile_dict, dataset_1_wsi_1_pos_dict)
for i,j in dataset_1_wsi_2_pos_dict.keys():
  adjacent_tile_dict = get_adjacent_tiles(i, j, dataset_1_wsi_2_pos_dict[(i,j)], adjacent_tile_dict, dataset_1_wsi_2_pos_dict)

422


In [9]:
# 2321d0fb53f7
adjacent_tile_dict['da1c6d6657e9']

{'B': '59642acc100e',
 'L': '733296a59c9b',
 'R': 'ae4641d3fb5f',
 'BL': None,
 'BR': '892146ef3207'}

In [10]:
[x for x in adjacent_tile_dict if adjacent_tile_dict[x]['B']=='9afac4fc65ef']

['1c8af4861691']

In [11]:
print(list(adjacent_tile_dict.keys())[200:220])
print(list(adjacent_tile_dict.keys()).index('cb96f7b681f8'))

['c9d0ac176add', 'c9dc3cfc0567', 'ca081412bad8', 'ca178713fdb2', 'ca57af0413ac', 'cb96f7b681f8', 'cbbdc5d83ec9', 'cbc4758c8c53', 'cd4b698ac682', 'cd6b90e3f4b3', 'ce27b0613895', 'cf921187e2bc', 'cfbf75947d43', 'cfe02239d1c7', 'd078cb53bbd5', 'd171145b9fb7', 'd1d485660263', 'd4e99e162f37', 'd6529e6e8602', 'd6d79e965902']
205


In [12]:
tgt_tile_dicts['c9d0ac176add']

[([[107, 67],
   [106, 67],
   [105, 67],
   [104, 67],
   [103, 67],
   [102, 67],
   [101, 67],
   [101, 66],
   [100, 66],
   [100, 65],
   [99, 65],
   [98, 65],
   [97, 65],
   [96, 65],
   [95, 65],
   [94, 65],
   [94, 64],
   [93, 64],
   [93, 63],
   [92, 63],
   [92, 62],
   [91, 62],
   [91, 61],
   [90, 61],
   [90, 60],
   [89, 60],
   [89, 59],
   [88, 59],
   [88, 58],
   [87, 58],
   [87, 57],
   [86, 57],
   [86, 56],
   [85, 56],
   [85, 55],
   [84, 55],
   [83, 55],
   [83, 54],
   [82, 54],
   [82, 53],
   [81, 53],
   [81, 52],
   [80, 52],
   [80, 51],
   [79, 51],
   [79, 50],
   [79, 49],
   [78, 49],
   [78, 48],
   [77, 48],
   [77, 47],
   [77, 46],
   [77, 45],
   [76, 45],
   [76, 44],
   [76, 43],
   [76, 42],
   [76, 41],
   [76, 40],
   [76, 39],
   [76, 38],
   [76, 37],
   [76, 36],
   [75, 36],
   [75, 35],
   [75, 34],
   [75, 33],
   [74, 33],
   [74, 32],
   [74, 31],
   [74, 30],
   [74, 29],
   [75, 28],
   [75, 28],
   [76, 27],
   [76, 27],
  

In [ ]:
import time
shifting_thresholds = [0.15,0.30,0.45,0.60,0.75,0.90]
print(f'Saving merged imgs and annots for {len(adjacent_tile_dict)} images...')
start_time = time.time()
for idx, img_id in enumerate(list(adjacent_tile_dict.keys())):
#   save_merged_imgs(input_imgs_dir, output_imgs_dir, img_id, adjacent_tile_dict[img_id], shifting_thresholds)
  save_merged_annotations(output_annots_dir, img_id, adjacent_tile_dict[img_id], shifting_thresholds, tgt_tile_dicts)
  if (idx+1) % 50 == 0:
    print(f'Took {float(time.time()-start_time)} seconds to finish saving data for {idx+1} images')

Saving merged imgs and annots for 422 images...
base image id: 0033bbc76b6b
base image id: 00656c6f2690
base image id: 00d75ad65de3
base image id: 00da70813521
base image id: 0596bfb19322
base image id: 06034408218a
base image id: 072f5307f243
base image id: 0788fc3be62e
base image id: 089a9e6be240
base image id: 093a52e9bc1d
base image id: 0a10b8716b30
base image id: 0b8029db1fb4
base image id: 0c54942878fa
base image id: 0d6c0cfee2db
base image id: 0d9d65340ef8
base image id: 0dce2b8d2c25
base image id: 0e0836cf1824
base image id: 0e8aed930dc6
base image id: 10162104cbaf
base image id: 1222b4306c01
base image id: 12ab9ac0fc55
base image id: 130d4d323ce4
base image id: 13aa34ced90d
base image id: 15e5df255b86
base image id: 15ed954c0cdf
base image id: 19551df15a1e
base image id: 1a54cda8f32d
base image id: 1a785cc0d167
base image id: 1bd726055ad8
base image id: 1c6c39a22324
base image id: 1c8af4861691
base image id: 1d8aa548c370
base image id: 1e0efbb8979f
base image id: 1e14b9acc9b9


In [ ]:
base_img_id = 'e7473674d813'
shifting_thresholds = [0.15,0.30,0.45,0.60,0.75,0.90]
save_merged_img_id_with_annot(output_imgs_dir, cleaned_output_annots_dir, vis_output_imgs_dir, base_img_id, adjacent_tile_dict[base_img_id], shifting_thresholds)

In [ ]:
base_img = load_img(vis_output_imgs_dir, f'{base_img_id}_full_annotated')
plt.imshow(base_img)
plt.show()
img = load_img(vis_output_imgs_dir, f'{base_img_id}_R_0.6_full_annotated')
# img_polygon_merged = load_img(vis_output_imgs_dir, f'{base_img_id}_BR_0.6_full_annotated_polygon_merged')
plt.imshow(img)
plt.show()
# plt.imshow(img_polygon_merged)
# plt.show()

In [ ]:
## Find polygons with less than 3 pairs of coordinates and remove them from the annotations
annot_files = os.listdir(output_annots_dir)
cleaned_output_annots_dir = f'{base_path}/dataset1_files/all_dataset1_annotations_merged_cleaned'
if not os.path.exists(cleaned_output_annots_dir):
    os.mkdir(cleaned_output_annots_dir)
for annot_file in annot_files:
  with open(f'{output_annots_dir}/{annot_file}', 'rb') as f:
    annots = pickle.load(f)
  cleaned_annots = [annot for annot in annots if len(annot['segmentation'][0]) >= 6]
  with open(f'{cleaned_output_annots_dir}/{annot_file}', 'wb') as f:
    pickle.dump(cleaned_annots, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## Find polygons with less than 3 pairs of coordinates and remove them from the annotations
cleaned_output_annots_dir = f'{base_path}/dataset1_files/all_dataset1_annotations_merged_cleaned'
annot_files = os.listdir(cleaned_output_annots_dir)
for annot_file in annot_files:
    with open(f'{cleaned_output_annots_dir}/{annot_file}', 'rb') as f:
        annots = pickle.load(f)
    img_id = annot_file.split('.pkl')[0]
    for annot in annots:
        segmentation = annot['segmentation'][0]
        new_segmentation = [[pt[0], pt[1]] for pt in zip(segmentation[::2], segmentation[1::2])]
        if len(new_segmentation) < 3:
            print(f'Segmentation error for img is {img_id}')

In [ ]:
data

In [ ]:
base_img_id = '0033bbc76b6b'
img_dir = f'{base_path}/dataset1_files/all_dataset1_imgs'
shifting_threshold = 0.15
img_id_1 = base_img_id
img_1 = cv2.imread(f'{img_dir}/{img_id_1}.png')
print(img_1.shape)
img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
vis_dict = {'base_image': img_1}
for k in adjacent_tile_dict[base_img_id]:
  img_id_2 = adjacent_tile_dict[base_img_id][k]
  if img_id_2 is not None:
    img_2 = cv2.imread(f'{img_dir}/{img_id_2}.png')
    img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB)
    vis_dict[f'shift_image_{k}_orig'] = img_2
    if k in ['R', 'B', 'L']:
      shifted_image = single_shift_img(img_1, img_2, shifting_threshold, k)
      print(shifted_image.shape, k)
      vis_dict[f'shift_image_{k}_shifted'] = shifted_image
    else:
      if k == 'BL' and adjacent_tile_dict[base_img_id]['B'] is not None and adjacent_tile_dict[base_img_id]['L'] is not None:
        img_2 = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['L']}.png")
        img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB)
        img_3 = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['B']}.png")
        img_3 = cv2.cvtColor(img_3, cv2.COLOR_BGR2RGB)
        img_4 = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id][k]}.png")
        img_4 = cv2.cvtColor(img_4, cv2.COLOR_BGR2RGB)
        shifted_image = double_shift_img(img_1, img_2, img_3, img_4, shifting_threshold, k)
        print(shifted_image.shape)
        vis_dict[f'shift_image_{k}_shifted'] = shifted_image
      elif k == 'BR' and adjacent_tile_dict[base_img_id]['B'] is not None and adjacent_tile_dict[base_img_id]['R'] is not None:
        img_2 = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['R']}.png")
        img_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2RGB)
        img_3 = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['B']}.png")
        img_3 = cv2.cvtColor(img_3, cv2.COLOR_BGR2RGB)
        img_4 = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id][k]}.png")
        img_4 = cv2.cvtColor(img_4, cv2.COLOR_BGR2RGB)
        shifted_image = double_shift_img(img_1, img_2, img_3, img_4, shifting_threshold, k)
        print(shifted_image.shape)
        vis_dict[f'shift_image_{k}_shifted'] = shifted_image
visualize(**vis_dict)

In [ ]:
base_img_id = '0033bbc76b6b'
img_dir = f'{base_path}/dataset1_files/all_dataset1_imgs'
shifting_threshold = 0.25
img_id_1 = base_img_id
img_1 = cv2.imread(f'{img_dir}/{img_id_1}.png')
img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
bottom_img = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['B']}.png")
bottom_img = cv2.cvtColor(bottom_img, cv2.COLOR_BGR2RGB)
left_img = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['L']}.png")
left_img = cv2.cvtColor(left_img, cv2.COLOR_BGR2RGB)
bottom_left_img = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['BL']}.png")
bottom_left_img = cv2.cvtColor(bottom_left_img, cv2.COLOR_BGR2RGB)
image_top = cv2.hconcat([left_img, img_1])
image_bottom = cv2.hconcat([bottom_left_img, bottom_img])
final_img = cv2.vconcat([image_top, image_bottom])
plt.imshow(final_img)
plt.show()
shifted_image = double_shift_img(img_1, left_img, bottom_img, bottom_left_img, shifting_threshold, 'BL')
plt.imshow(shifted_image)
plt.show()

In [ ]:
base_img_id = '0033bbc76b6b'
img_dir = f'{base_path}/dataset1_files/all_dataset1_imgs'
shifting_threshold = 0.5
img_id_1 = base_img_id
img_1 = cv2.imread(f'{img_dir}/{img_id_1}.png')
img_1 = cv2.cvtColor(img_1, cv2.COLOR_BGR2RGB)
bottom_img = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['B']}.png")
bottom_img = cv2.cvtColor(bottom_img, cv2.COLOR_BGR2RGB)
right_img = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['R']}.png")
right_img = cv2.cvtColor(right_img, cv2.COLOR_BGR2RGB)
bottom_right_img = cv2.imread(f"{img_dir}/{adjacent_tile_dict[base_img_id]['BR']}.png")
bottom_right_img = cv2.cvtColor(bottom_right_img, cv2.COLOR_BGR2RGB)
image_top = cv2.hconcat([img_1, right_img])
image_bottom = cv2.hconcat([bottom_img, bottom_right_img])
final_img = cv2.vconcat([image_top, image_bottom])
plt.imshow(final_img)
plt.show()
shifted_image = double_shift_img(img_1, right_img, bottom_img, bottom_right_img, shifting_threshold, 'BR')
plt.imshow(shifted_image)
plt.show()